In [1]:
import sys
import numpy as np
# PIL -visulization tools,Image-read/write, ImageTK-user interface
from PIL import Image, ImageTk
#bit-bytes problem solve; encoding to decoding, hexa-decimal
from io import BytesIO

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# requests for client/user
import requests
#Tk-user interface, Canvas-Area, Label-image seting position, NW-calling image
from tkinter import Tk, Label, Canvas, Entry, Button, NW


#load_model: save model then load model
from tensorflow.keras.models import Sequential,load_model

# image layers, dense-fully connected, Conv2D-Basic 2D CNN, Flatten-Create bridge
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# text for NLP, load_img, img_to_array-convert image to array
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
#image_utils-image raw processing
from tensorflow.keras.preprocessing import image as image_utils

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
# empty space creat
model = Sequential()

#input layer,Conv2D-create layer for image,  32-number of node, (3,3)-kernel size,input_shape-RGB image(width, height, color)
model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu"))
#parameters: MaxPooling2D:pool_size =(2,2)-
model.add(MaxPooling2D(pool_size = (2, 2)))

#Hidden layer, 64-node assending order of input layer node
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

# create the bridge by Flatten
model.add(Flatten())

#When we will create bridge layer, we don't connect with output layer directly; node should be same with previous layer
model.add(Dense(128, activation="relu"))

#Output layer: softmax-more than 2 output
model.add(Dense(2, activation="sigmoid"))

Metal device set to: Apple M1


2021-11-30 15:21:35.251934: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-30 15:21:35.252041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
#compile optimizer, loss function, metrics for accuracy
model.compile(optimizer="adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

In [4]:
#data process: rescale image:convert grey image from RGB, shear_range: shape position change by angle wise like image augmentation,zoom_range: zooming data,horizontal_flip: horizontal/vertical_flip  
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip= True)

In [5]:
#data process
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [6]:

#flow_from_directory: dataset location, target size:unknown size, maximum size, batch size: relation with validation split and total number of size,class_mode:categorical
training_datasets = train_datagen.flow_from_directory("dataset_cats/training_set/training_set/", target_size= (128, 128), batch_size = 32, class_mode = "categorical")
testing_datasets = test_datagen.flow_from_directory("dataset_cats/test_set/test_set/", target_size=(128, 128), batch_size=32, class_mode="categorical")

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [7]:
#fit the model. steps_per_epoch: total=2400/3=800, 32-batch, spochs=50, validation_steps=total number of images/3(classes)=200
model.fit(training_datasets, steps_per_epoch=4002/32, epochs=10, validation_data=testing_datasets,validation_steps =  1010/32)


2021-11-30 15:21:35.857034: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 15:21:35.857154: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-30 15:21:35.988374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
126/125 [==============================] - ETA: 0s - loss: 0.6897 - accuracy: 0.5375

2021-11-30 15:21:48.600234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 14s 111ms/step - loss: 0.6897 - accuracy: 0.5375 - val_loss: 0.6761 - val_accuracy: 0.5605
Epoch 2/10
125/125 [==============================] - 14s 109ms/step - loss: 0.6530 - accuracy: 0.6079 - val_loss: 0.6669 - val_accuracy: 0.6221
Epoch 3/10
125/125 [==============================] - 14s 109ms/step - loss: 0.6320 - accuracy: 0.6340 - val_loss: 0.6674 - val_accuracy: 0.5986
Epoch 4/10
125/125 [==============================] - 14s 108ms/step - loss: 0.6036 - accuracy: 0.6757 - val_loss: 0.5671 - val_accuracy: 0.7188
Epoch 5/10
125/125 [==============================] - 14s 108ms/step - loss: 0.5738 - accuracy: 0.6957 - val_loss: 0.5496 - val_accuracy: 0.7197
Epoch 6/10
125/125 [==============================] - 14s 109ms/step - loss: 0.5608 - accuracy: 0.7166 - val_loss: 0.5288 - val_accuracy: 0.7432
Epoch 7/10
125/125 [==============================] - 14s 111ms/step - loss: 0.5357 - accuracy: 0.7363 - val_loss: 0.5085 - val_accuracy: 0.7

In [8]:
#Save the model
model.save("model_cats/model.h5")
model.save_weights("model_cats/model_weights.h5")

In [9]:
#load the model
image_width, image_height = 128, 128
#create the model
model_path = "model_cats/model.h5"
#create model for weight
model_wight_path = "model_cats/model_weights.h5"

final_model = load_model(model_path) 
final_model.load_weights(model_wight_path)

In [21]:
url=''
window=Tk()
window.title("Image classification using CNN")#plt.title()
window.geometry("800x800")
label = Label(window,text="Please Enter your Custom URL",font=("Halvetica",16))
label.pack()

def Enter():
    global url
    label.configure()#why
    url=(User_input.get())
    print(url)
    
    response = requests.get(url)
    test_image=Image.open(BytesIO(response.content))
    put_image=test_image.resize((400,400))#source image
    test_image=test_image.resize((128,128))

    img = ImageTk.PhotoImage(put_image)
    pic = Label(image=img)
    pic.pack()# merge the image

    pic.image=img
    test_image=image_utils.img_to_array(test_image)
    test_image= np.expand_dims(test_image,axis=0)

    results= final_model.predict_on_batch(test_image)

    if results[0][0]==1:
        res="cats"
    elif results[0][1]==1:
        res="dogs"

    output= Label(window,text= "Predicted Results:"+res,font=("Halvetica",16))
    output.pack()
    #window.mainloop()
    print(res)

In [22]:
User_input=Entry()
User_input.pack()
button=Button(window,text="Detect Image",font=("Halvetica",16),command=Enter)
button.pack()
window.mainloop()

http://learnwebcode.com/images/lessons/insert-image-funny-dog.jpg
dogs
